# RAG System Experimentation Notebook

Interactive notebook for running experiments with verbose output and real-time analysis.

## Features

1. **Verbose output captured inline** - See router/planner/retriever decisions in real-time
2. **Detailed JSON output** - Full config, chunk indices, distances for reproducibility
3. **Interactive analysis** - Visualize and compare results immediately
4. **Grid search support** - Test 270 variations efficiently

## Setup

In [ ]:
import pandas as pd
import json
from pathlib import Path
import sys

# Import experiment runner
from experiment_runner_enhanced import EnhancedExperimentRunner, VariationConfig
from src.agents import OrchestratorConfig, RouterConfig, PlannerConfig, RetrieverConfig

print("✓ Imports successful")

## Define Experiment Variations

Customize your grid search here. Example shows comprehensive parameter sweep.

In [ ]:
class CustomExperimentRunner(EnhancedExperimentRunner):
    """Custom runner with grid search variations."""
    
    def create_variations(self):
        """
        Create grid search variations.
        
        Parameters tested:
        - chunk_size: 400, 800, 1200, 1600, 2000
        - top_k_per_step: 1, 3, 5
        - chunking_strategy: page, document
        - top_k_docs: 1, 3, 5
        - expand_context: 0, 1, 2
        
        Total: 5 * 3 * 2 * 3 * 3 = 270 variations
        """
        variations = []
        
        # Grid search
        for i in range(5):
            chunk_size = (i + 1) * 400
            
            for top_k_per_step in range(1, 6, 2):  # 1, 3, 5
                for typ in ["page", "document"]:
                    for docs in range(1, 6, 2):  # 1, 3, 5
                        for wdw in [0, 1, 2]:
                            variations.append(
                                VariationConfig(
                                    name=f"chunk_{chunk_size}_topk_{top_k_per_step}_window{wdw}_docs{docs}_{typ}",
                                    description=(
                                        f"chunk_size={chunk_size}, "
                                        f"top_k_per_step={top_k_per_step}, "
                                        f"top_k_docs={docs}, "
                                        f"chunking_strategy={typ}"
                                    ),
                                    orchestrator_config=OrchestratorConfig(
                                        model="llama3.2",
                                        temperature=0.0,
                                        max_answer_tokens=4096,
                                        router_config=RouterConfig(
                                            model="llama3.2",
                                            top_k_docs=docs,
                                            temperature=0.0,
                                        ),
                                        planner_config=PlannerConfig(
                                            model="llama3.2",
                                            temperature=0.0,
                                        ),
                                        retriever_config=RetrieverConfig(
                                            top_k_per_step=top_k_per_step,
                                            chunk_size=chunk_size,
                                            chunk_overlap=200,
                                            expand_context=wdw,
                                            chunking_strategy=typ,
                                            use_hybrid=True,  # Enable hybrid search (best performer)
                                            hybrid_alpha=0.5  # Adaptive at runtime
                                        ),
                                    ),
                                )
                            )
        
        return variations

# Create runner
runner = CustomExperimentRunner()
variations = runner.create_variations()
print(f"✓ Created {len(variations)} variations")

## Test Single Variation (Quick Check)

Test one variation first to ensure everything works before running full grid search.

In [ ]:
# Load test questions
questions_df = pd.read_csv("artifacts/questions.csv")
print(f"Loaded {len(questions_df)} questions\n")

# Test first variation on first question
test_question = questions_df.iloc[0]
test_variation = variations[0]

print(f"Testing: {test_variation.name}")
print(f"Question: {test_question['question'][:100]}...\n")

# Run with verbose=True to see all intermediate steps
result = runner.run_single_question_detailed(
    question_id=test_question['id'],
    question=test_question['question'],
    expected_output=test_question['expected_output'],
    pdf_folder=test_question['PDF Folder'],
    variation=test_variation,
    verbose=True  # Shows all agent decisions
)

print("\n" + "="*70)
print("RESULT SUMMARY")
print("="*70)
print(f"Exact match: {result.exact_match}")
print(f"Partial match: {result.partial_match_score:.2%}")
print(f"Execution time: {result.execution_time_seconds}s")
print(f"Total chunks: {result.total_chunks_retrieved}")

## Run Full Experiment Suite

⚠️ **Warning**: 270 variations × 2 questions = 540 runs (~10-20 hours)

Consider:
- Reducing variations for initial testing
- Running overnight
- Using subset of questions first

In [ ]:
# Option 1: Run ALL variations (long-running)
# results_df = runner.run_all_experiments(
#     verbose=True,
#     clean_collections=True,
#     save_detailed_json=True
# )

# Option 2: Run subset for testing (recommended first)
class SubsetRunner(EnhancedExperimentRunner):
    """Run subset of variations for quick testing."""
    
    def create_variations(self):
        # Test just a few key configurations
        variations = []
        
        # Test: chunk size impact (fix other params)
        for chunk_size in [400, 800, 1200, 1600, 2000]:
            variations.append(
                VariationConfig(
                    name=f"chunk_{chunk_size}",
                    description=f"chunk_size={chunk_size} (baseline config)",
                    orchestrator_config=OrchestratorConfig(
                        model="llama3.2",
                        temperature=0.0,
                        max_answer_tokens=4096,
                        router_config=RouterConfig(
                            model="llama3.2",
                            top_k_docs=3,
                            temperature=0.0,
                        ),
                        planner_config=PlannerConfig(
                            model="llama3.2",
                            temperature=0.0,
                        ),
                        retriever_config=RetrieverConfig(
                            top_k_per_step=5,
                            chunk_size=chunk_size,
                            chunk_overlap=200,
                            expand_context=2,
                            chunking_strategy="page",
                            use_hybrid=True,
                            hybrid_alpha=0.5
                        ),
                    ),
                )
            )
        
        return variations

# Run subset (5 variations × 2 questions = 10 runs)
subset_runner = SubsetRunner()
results_df = subset_runner.run_all_experiments(
    verbose=True,
    clean_collections=True,
    save_detailed_json=True
)

print("\n✓ Experiment complete!")

## Analyze Results

Quick analysis of experiment results.

In [ ]:
# Summary statistics
print("="*70)
print("RESULTS SUMMARY")
print("="*70)
print(f"\nTotal runs: {len(results_df)}")
print(f"Variations tested: {results_df['variation_name'].nunique()}")
print(f"Questions: {results_df['question_id'].nunique()}")

# Performance by variation
print("\n" + "-"*70)
print("Performance by Variation")
print("-"*70)

summary = results_df.groupby('variation_name').agg({
    'exact_match': 'sum',
    'partial_match_score': 'mean',
    'execution_time_seconds': 'mean',
    'total_chunks_retrieved': 'mean'
}).round(2)

summary.columns = ['Exact Matches', 'Avg Partial', 'Avg Time (s)', 'Avg Chunks']
summary = summary.sort_values('Avg Partial', ascending=False)

print(summary)

# Best performer
best_var = summary.index[0]
print(f"\n🏆 Best performer: {best_var}")
print(f"   Avg partial match: {summary.loc[best_var, 'Avg Partial']:.2%}")
print(f"   Avg time: {summary.loc[best_var, 'Avg Time (s)']:.1f}s")

## Analyze Detailed JSON

Examine chunk-level details for deep analysis.

In [ ]:
# Load detailed JSON (most recent)
results_dir = Path("results")
json_files = sorted(results_dir.glob("detailed_results_*.json"))

if json_files:
    latest_json = json_files[-1]
    print(f"Loading: {latest_json}")
    
    with open(latest_json) as f:
        detailed_results = json.load(f)
    
    print(f"\nLoaded {len(detailed_results['results'])} detailed results")
    
    # Example: Examine first result
    first_result = detailed_results['results'][0]
    
    print("\n" + "="*70)
    print("EXAMPLE DETAILED RESULT")
    print("="*70)
    print(f"\nQuestion: {first_result['question_id']}")
    print(f"Variation: {first_result['variation_name']}")
    print(f"\nRouter selected: {len(first_result['router_selected_docs'])} docs")
    for doc in first_result['router_selected_docs']:
        print(f"  - {doc[:60]}...")
    
    print(f"\nPlanner strategy: {first_result['planner_strategy']}")
    print(f"Number of steps: {first_result['planner_num_steps']}")
    
    print(f"\nRetrieval steps:")
    for step in first_result['retrieval_steps']:
        print(f"\n  Step {step['step_number']}: {step['description']}")
        print(f"  Query: {step['query']}")
        print(f"  Retrieved: {step['num_chunks']} chunks")
        
        # Show first chunk details
        if step['chunks']:
            chunk = step['chunks'][0]
            print(f"  Example chunk:")
            print(f"    - Index: {chunk['chunk_index']}")
            print(f"    - Source: {chunk['source_file'][:40]}...")
            print(f"    - Page: {chunk['page_number']}")
            print(f"    - Distance: {chunk['distance']:.3f}")
            print(f"    - Text preview: {chunk['text_preview'][:100]}...")
    
    print(f"\nMetrics:")
    print(f"  Exact match: {first_result['exact_match']}")
    print(f"  Partial match: {first_result['partial_match_score']:.2%}")
    print(f"  Execution time: {first_result['execution_time_seconds']}s")
else:
    print("No detailed JSON results found. Run experiments first.")

## Compare Configurations

Compare which chunks were retrieved for different configurations.

In [ ]:
# Compare chunk retrieval for EF_1 across different variations
if 'detailed_results' in locals():
    ef1_results = [r for r in detailed_results['results'] if r['question_id'] == 'EF_1']
    
    print("="*70)
    print("CHUNK RETRIEVAL COMPARISON - EF_1")
    print("="*70)
    
    for result in ef1_results[:3]:  # Show first 3 variations
        print(f"\n{result['variation_name']}:")
        print(f"  Partial match: {result['partial_match_score']:.2%}")
        print(f"  Total chunks: {result['total_chunks_retrieved']}")
        
        # Show chunk indices retrieved
        chunk_indices = []
        for step in result['retrieval_steps']:
            for chunk in step['chunks']:
                chunk_indices.append(chunk['chunk_index'])
        
        print(f"  Chunk indices: {sorted(set(chunk_indices))}")
    
    print("\nℹ️  This shows which chunks each configuration retrieved.")
    print("   Compare to understand why some configs perform better.")

## Export for Further Analysis

Export results for analysis in other tools.

In [ ]:
# Export summary to CSV for Excel/other tools
output_path = "results/experiment_summary.csv"
results_df.to_csv(output_path, index=False)
print(f"✓ Exported summary to: {output_path}")

# Create pivot table for heatmap analysis
if len(results_df) > 0:
    pivot = results_df.pivot_table(
        values='partial_match_score',
        index='variation_name',
        columns='question_id',
        aggfunc='mean'
    )
    
    pivot_path = "results/experiment_pivot.csv"
    pivot.to_csv(pivot_path)
    print(f"✓ Exported pivot table to: {pivot_path}")
    print("\nPivot table preview:")
    print(pivot)

## Notes

- **Verbose output**: Set `verbose=True` to see all agent decisions inline
- **Detailed JSON**: Contains full config, chunk indices, and intermediate results
- **Reproducibility**: Use JSON config to exactly recreate any run
- **Analysis**: Compare chunk indices to understand performance differences

## Next Steps

1. Run subset experiments to identify promising parameter ranges
2. Analyze detailed JSON to understand retrieval patterns
3. Run full grid search on promising ranges
4. Compare chunk-level details for best vs worst performers
5. Document findings and update recommendations